# DEL 2 API fra yr.no

In [ ]:
# Importer bibliotekene vi trenger
import requests
import json
import pandas as pd
from datetime import datetime

# For automatiske tester trenger vi pytest og ipytest
import ipytest
ipytest.autoconfig()

## Oppgave 3: Introduksjon til Yr.no API

Nå skal vi begynne å arbeide med ekte værdata! Yr.no tilbyr gratis værdata gjennom sitt API.

**API dokumentasjon:** https://api.met.no/weatherapi/locationforecast/2.0/documentation
Gå inn på lenken over og ha denne tilgjengelig mens du jobber videre. Siden inneholder API-spesifiksjonen, og forteller utviklere hvordan de kan bruke yr sin locationforecast api. 

**Base-adresse:** REST APIer bruker å ha en base-adresse, som kommer først for alle uri-er knyttet til APIet. 
For locations apiet til yr er baseadressen `https://api.met.no/weatherapi/locationforecast/2.0/`. Ofte, som her,
inkluderer base-uri et versjonsnummer. Slik kan APIet oppdateres samtidig som den gamle versjonen er tilgjengelig,
slik at utviklere har tid til å oppdatere sine programmer til å støtte spesifikasjon i ny versjon av APIet.

> Yr har flere APIer som du kan utforske på https://api.met.no
> Vi skal bruke loactionforecast som eksempel her

La oss prøve et enkelt kall til APIet, med uri `https://api.met.no/weatherapi/locationforecast/2.0/status`. 
Dette skal gi oss en status-beskjed, og kan brukes for å sjekke om vi klarer å koble oss opp mot APIet.

In [ ]:
response_3a = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/status')
print("Status kode:", response_3a.status_code)

Oops.. Her fikk vi en feil statuskode - 403

---

### Oppgave 3a
Hva betyr denne statuskoden?

_Skriv svaret ditt her_

---

Hvis vi ser i dokumentasjonen, står det i delen 
[AUTHENTICATION](https://api.met.no/weatherapi/locationforecast/2.0/documentation#AUTHENTICATION) 
at en er nødt til å bruke en unik identifikator i User-Agent header-verdi for at ikke forespørselen
skal avvises.

La oss sjekke hvordan våre header-verdier så ut fra requesten vi sendte over

> Tips:
> Informasjon om sendte forespørsler kan hentes ut fra respons-variabelen
> med egenskapen `request` (eks: `respons_variabel.request`)

In [ ]:
# Her skriver vi ut headere som ble sendt med forespørselen
# Kjør cellen og se på output
print("\nRequest headers:")
request_headers = response_3a.request.headers
for key, value in request_headers.items():
    print(f"{key}: {value}")

---

### Oppgave 3b

Hva er verdien til headeren _User-Agent_?

_Skriv svaret ditt her_

---

Verdien på _User-Agent_ oppfyller ikke vilkårene til i API-spesifikasjonen. La oss prøve igjen der vi setter en unik verdi for User-Agent:

In [ ]:
# Kjør cellen under for å gjøre et nytt kall med en gyldig User-Agent header

# Sett en unik User-Agent header som beskrevet i dokumentasjonen
headers = {
    'User-Agent': 'TestApp/1.0 (it-test@sanpro.no)'
}

# Gjør forespørselen med de nye headerne
response = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/status', headers=headers)

print("Status kode:", response.status_code)

# Skriv ut headere som ble sendt med forespørselen
print("\nRequest headers:")
print(json.dumps(dict(response.request.headers), indent=2))

# Hvis statuskoden er 200 (ok), skriv ut JSON-responsen
if response.status_code == 200:
    data = response.json()
    print("\nRespons data:")
    print(json.dumps(data, indent=2))
else:
    print("Noe gikk galt med forespørselen.")

Nå fikk vi et gyldig svar! 

---

### Oppgave 3c

Hva inneholdt responsen av data?

_Skriv svaret ditt her_

---

> Tips ISO 8601
> Les om standarden ISO 8601 for dato og tid formatert som tekst på [Wikipedia](https://en.wikipedia.org/wiki/ISO_8601)

Som vi nå har erfart er det lurt å sjekke dokumentasjon på API før bruk, og også vilkår. Les yr sine [Terms of Service her](https://api.met.no/doc/TermsOfService)

**Viktige punkter om Yr.no API:**
- Du må identifisere applikasjonen din med en "User-Agent" header
- Du kan ikke sende for mange forespørsler (maks 20 per sekund)
- API-et krever geografiske koordinater (bredde- og lengdegrad)

## Oppgave 4: Et enkelt kall til locationforecast

Siden det er noen verdier vi kommer til å bruke flere ganger videre, kan det være lurt å lagre disse til noen variabler først. 

Kjør cellen under for at variablene som defineres skal være tilgjengelig senere.

In [ ]:
# Sett opp korrekte headers for Yr.no API
headers = {
    'User-Agent': 'TestApp/1.0 (it-test@sanpro.no)'  # Bytt gjerne ut med din e-post
}

# URL for Yr.no API
base_url = "https://api.met.no/weatherapi/locationforecast/2.0/"
base_url_compact = base_url + "compact"
base_url_complete = base_url + "complete"

# Utskrift av base-verdiene
print("API URL:", base_url)
print("API url for compact metoden:", base_url_compact)
print("API url for complete metoden:", base_url_complete)
print("Headers:", headers)

_Kjør cellen under for å teste om variablene er satt opp riktig_

In [ ]:
%%ipytest
# Test at variablene er satt opp riktig
def test_api_setup():
    assert base_url == "https://api.met.no/weatherapi/locationforecast/2.0/"
    assert base_url_compact == "https://api.met.no/weatherapi/locationforecast/2.0/compact"
    assert base_url_complete == "https://api.met.no/weatherapi/locationforecast/2.0/complete"
    assert 'User-Agent' in headers
    assert headers['User-Agent'] != 'python-requests/'  # Sjekk at User-Agent er unik

Da kan vi gjøre vårt første ordentlige kall til APIet:

In [ ]:

# Koordinater for Hana, Sandnes
lat = 58.854  # Breddegrad (N-S)
long = 5.767  # Lengdegrad (Ø-V)

# Bygg URL for API-forespørsel
url = f"{base_url_compact}?lat={lat}&lon={long}"

# Gjennomfør forespørselen
response = requests.get(url, headers=headers)
print("Status kode:", response.status_code)
if response.status_code == 200:
    print("Respons data (formatert):")
    data = response.json()
    print(json.dumps(data, indent=2)[:500], "...")  # Print de første 500 tegnene av responsen
else:
    print("Noe gikk galt med forespørselen.")

---

### Oppgave 4a - Lag din første værdata-forespørsel

Velg en lokasjon og hent værdata for denne lokasjonen med et enkelt API-kall. Skriv koden din under. 

**Lagre responsen i en variabel med navn `response_4a`**

_Tips: Bruk google maps til å finne koordinatene_

In [ ]:
# Skriv din kode her
# OBS! Lagre responsen i en variabel med navn response_4a, slik at vi kan bruke den i neste oppgave.


# Koordinater for Hana, Sandnes
lat = 58.854  # Breddegrad (N-S)
long = 5.767  # Lengdegrad (Ø-V)

# Bygg URL for API-forespørsel
url = f"{base_url_compact}?lat={lat}&lon={long}"

response_4a = requests.get(url, headers=headers)

_Kjør cellen under for å sjekke at du gjorde oppgaven over riktig_

In [ ]:
%%ipytest

def test_response_4a():
    assert 'response_4a' in globals(), "Variabelen response_4a finnes ikke."
    assert response_4a.status_code == 200, "Responsen i variabelen response_4a har ikke statuskode 200."


***

## Oppgave 5: Utforsk værdata-strukturen

Før vi kan bruke dataene, må vi forstå hvordan de er organisert.

In [ ]:
# Konverter til JSON og lagre til fil
if response_4a.status_code == 200:
    data_task4 = response_4a.json()
    with open('weather_data.json', 'w') as f:
        json.dump(data_task4, f, indent=2)
    print("Data lagret til weather_data.json")
else:
    print("Feil! Respons ugyldig.")
    print("Sjekk om du har gjennomført forrige oppgave korrekt, og kjørt cellen.")
    print("OG at du har lagret responsen i en variabel med navn response_4a.")

---

### Oppgave 5a

Åpne filen `weather_data.json` og se gjennom verdiene. Filen er ganske stor, men du vil se at strukturen gjentas etter 50-100 linjer, så du trenger bare å forstå starten for å skjønne hele innholdet. 

- Hvilke hovedegenskaper (1. nivå) er det i json-filen? (det skal være tre stk, navngi dem)
- Hvilke egenskaper finnes under "properties" ?
- Hvor finner du de faktiske værdataene i filen?

_Skriv svaret ditt her_

***

### Oppgave 5b - Se nærmere på strukturen til en enkelt prognose

Fullfør funksjonene `get_properties` og `get_timeseries` under. Husk å returnere en verdi som siste linje av funksjonen:

```python
def get_properties(data_locationforecast=data_task4):
    # ... din logikk
    return variabel_navn # Returner en verdi til slutt
```


In [ ]:
def get_properties(data_locationforecast=data_task4):
    """Hent ut properties fra responsen fra locationforecast compact API-et"""
    pass # Erstatt denne linjen med din kode
    return data_locationforecast['properties']

def get_timeseries(data_locationforecast=data_task4):
    """Hent ut prognoser fra responsen fra locationforecast compact API-et."""
    pass # Erstatt denne linjen med din kode
    return data_locationforecast['properties']['timeseries']


Kjør cellen over etter du har fullført funksjonene. Du vil se en grønn hake ✅, men det kommer ingen output. Det er ok, og forventet. 

Etter du har kjørt cellen over (som laster funksjonsreferansene inn i python-interpreterens minne), kan du bruke testen under for å sjekke om løsningen din er gyldig.

_Kjør cellen under for å sjekke løsningen din_

In [ ]:
%%ipytest 

def test_get_properties():
    properties = get_properties()
    assert isinstance(properties, dict), "get_properties skal returnere en dictionary."
    assert 'timeseries' in properties, "properties skal inneholde nøkkelen 'timeseries'."
    assert isinstance(properties['timeseries'], list), "properties['timeseries'] skal være en liste."
    
def test_get_timeseries():
    timeseries = get_timeseries()
    assert isinstance(timeseries, list), "get_timeseries skal returnere en liste."
    assert len(timeseries) > 0, "get_timeseries skal returnere en ikke-tom liste."
    first_entry = timeseries[0]
    assert 'time' in first_entry, "Hver entry i timeseries skal inneholde nøkkelen 'time'."
    assert 'data' in first_entry, "Hver entry i timeseries skal inneholde nøkkelen 'data'."


***

### Oppgave 5c: Utforsk prognosens datastruktur

Kjør cellen under og se på utskriften

In [ ]:
# Utforsk en enkelt prognose i detalj
def print_timeseries_keys():
    """Skriv ut alle nøklene i en enkelt prognose i timeseries-listen"""
    prognoser = get_timeseries()
    first_item = prognoser[0]
    def print_keys(d, prefix=''):
        for key, value in d.items():
            print(f"{prefix}{key}")
            if isinstance(value, dict):
                print_keys(value, prefix + '  ')
    print_keys(first_item)
    
print_timeseries_keys()

__Forklar utskriften over__

- Hvilken informasjon finnes i et timeseries objekt?

_Skriv ditt svar her_

***

## Oppgave 6: Lag en klasse for værdata

Nå som vi forstår datastrukturen, kan vi enklere hente ut data vi ønsker.

### Oppgave 6a - Lag en klasse for værdata

Lag en klasse, `Prognosis`, som

***

## Oppgave 6: Lag en klasse for værdata

Nå som vi forstår datastrukturen, kan vi lage en klasse for å jobbe med værdataene på en enkel måte.

### Oppgave 6a - Lag en enkel Prognosis-klasse

Lag en klasse `Prognosis` som:
- Tar inn et element fra timeseries-listen i konstruktøren
- Lagrer `time` som en instansvariabel av typen datetime
- Lagrer følgende værdata som instansvariabler med navnene:
    - Lufttrykk: `air_pressure`
    - Temperatur: `air_temperature`
    - Skydekke: `cloud_fraction`
    - Fuktighet: `humidity`
    - Vindretning: `wind_direction`
    - Vindhastighet: `wind_speed`

**Eksempel på bruk:**
```python
prognoser = get_timeseries()
første_prognose = Prognosis(prognoser[0])
print(første_prognose.time)  # Skriver ut tidspunktet
```

In [ ]:
class Prognosis:
    """Klasse for å representere en værprognose fra Yr.no API"""
    
    def __init__(self, timeseries_entry):
        """Initialiserer en Prognosis-instans fra et timeseries-objekt"""
        pass  # Erstatt denne linjen med din kode
        self.time = datetime.fromisoformat(timeseries_entry['time'])
        self.air_pressure = timeseries_entry['data']['instant']['details'].get('air_pressure_at_sea_level')
        self.air_temperature = timeseries_entry['data']['instant']['details'].get('air_temperature')
        self.cloud_fraction = timeseries_entry['data']['instant']['details'].get('cloud_fraction')
        self.humidity = timeseries_entry['data']['instant']['details'].get('relative_humidity')
        self.wind_direction = timeseries_entry['data']['instant']['details'].get('wind_from_direction')
        self.wind_speed = timeseries_entry['data']['instant']['details'].get('wind_speed')


_Kjør cellen under for å teste klassen din_

In [ ]:
%%ipytest

def test_prognosis_init():
    prognoser = get_timeseries()
    prognose = Prognosis(prognoser[0])
    
    assert isinstance(prognose.time, datetime), "prognose.time skal være av typen datetime."
    assert isinstance(prognose.air_pressure, (float, type(None))), "prognose.air_pressure skal være float eller None."
    assert isinstance(prognose.air_temperature, (float, type(None))), "prognose.air_temperature skal være float eller None."
    assert isinstance(prognose.cloud_fraction, (float, type(None))), "prognose.cloud_fraction skal være float eller None."
    assert isinstance(prognose.humidity, (float, type(None))), "prognose.humidity skal være float eller None."
    assert isinstance(prognose.wind_direction, (float, type(None))), "prognose.wind_direction skal være float eller None."
    assert isinstance(prognose.wind_speed, (float, type(None))), "prognose.wind_speed skal være float eller None."


***

### Oppgave 6b - bruk Prognosis

Bruk klassen du lagde, `Prognosis`, til å holde værdata hentet fra API. Bruk `get_timeseries()` for å hente en tidsserie, og last den første prognosen inn i et nytt `Prognosis` objekt med variabelnavn `first_prognosis`

In [ ]:
time_series = get_timeseries()
first_element = time_series[0]
first_prognosis = Prognosis(first_element)

_Kjør cellen under for å sjekke svaret ditt_

In [ ]:
%%ipytest

def test_first_prognosis():
    assert isinstance(first_prognosis, Prognosis), "first_prognosis skal være en instans av Prognosis."
    expected_timestamp = datetime.fromisoformat(get_timeseries()[0]['time'])
    assert first_prognosis.time == expected_timestamp, "first_prognosis.time har ikke riktig verdi."

Bruk `first_prognosis` for å skrive ut (print) temperatur og vindhastighet

In [ ]:
def print_weather_info():
    """Skriv ut temperatur og vindhastighet fra en Prognosis-instans"""
    pass # Erstatt denne linjen med din kode
    print(f"Temperatur: {first_prognosis.air_temperature} °C")
    print(f"Vindhastighet: {first_prognosis.wind_speed} m/s")

print_weather_info()

In [ ]:
%%ipytest

def test_print_weather_info(capsys):
    print_weather_info()
    captured = capsys.readouterr()
    prognosis = get_timeseries()[0]['data']['instant']['details']
    assert str(prognosis['air_temperature']) in captured.out, "Temperaturdata mangler i utskriften."
    assert str(prognosis['wind_speed']) in captured.out, "Vindhastighetsdata mangler i utskriften."

---

### Oppgave 6c - Lag en enkel Timeseries-klasse



In [ ]:
class Timeseries:
    """Klasse for å representere en tidsserie fra Yr.no API"""
    
    def __init__(self, timeseries):
        """Initialiserer en Timeseries-instans fra et timeseries-objekt"""
        self.entries = [Prognosis(entry) for entry in timeseries['properties']['timeseries']]
    
    def get_temperature(self):
        """Henter lufttemperatur i celsius"""
        pass  # Erstatt denne linjen med din kode
    
    def get_wind_speed(self):
        """Henter vindhastighet i m/s"""
        pass  # Erstatt denne linjen med din kode


_Kjør cellen under for å teste metodene dine_

In [ ]:
%%ipytest

def test_prognosis_methods():
    prognoser = get_timeseries()
    prognose = Prognosis(prognoser[0])
    
    # Test at metodene returnerer tall
    temp = prognose.get_temperature()
    wind = prognose.get_wind_speed()
    
    assert isinstance(temp, (int, float)), "get_temperature() skal returnere et tall"
    assert isinstance(wind, (int, float)), "get_wind_speed() skal returnere et tall"
    
    # Test at verdiene er rimelige
    assert -50 < temp < 50, "Temperatur skal være mellom -50 og 50°C"
    assert 0 <= wind < 100, "Vindhastighet skal være mellom 0 og 100 m/s"


---

La oss teste klassen vår med ekte data:

In [ ]:
# Hent prognoser og lag Prognosis-objekter
prognoser = get_timeseries()

# Vis de 5 første prognosene
print("De 5 første værmeldingene:\n")
for i in range(5):
    p = Prognosis(prognoser[i])
    print(f"{p.time}: {p.get_temperature()}°C, vind {p.get_wind_speed()} m/s")

---

### Oppgave 6c - Finn høyeste og laveste temperatur

Skriv kode som:
1. Går gjennom alle prognoser i timeseries
2. Finner den høyeste temperaturen
3. Finner den laveste temperaturen
4. Skriver ut resultatene

**Tips:** Bruk en løkke og to variabler for å holde styr på min og max temperatur.

In [ ]:
# Skriv din kode her
prognoser = get_timeseries()

# Initialiser variabler for å holde styr på min/max
# ... din kode

# Gå gjennom alle prognoser
# ... din kode

# Skriv ut resultatet
# ... din kode


***

## Oppgave 7: Visualiser værdata med pandas

Nå skal vi bruke pandas til å analysere og visualisere værdataene på en mer strukturert måte.

### Oppgave 7a - Lag en DataFrame med værdata

Lag en pandas DataFrame som inneholder:
- Tidspunkt (time)
- Temperatur (temperature)
- Vindhastighet (wind_speed)

**Tips:** Bruk en liste med dictionaries og `pd.DataFrame()`

In [ ]:
# Lag en liste med værdata
prognoser = get_timeseries()
weather_data = []

for entry in prognoser:
    p = Prognosis(entry)
    weather_data.append({
        'time': p.time,
        'temperature': p.get_temperature(),
        'wind_speed': p.get_wind_speed()
    })

# Lag DataFrame
df = pd.DataFrame(weather_data)

# Konverter time til datetime
df['time'] = pd.to_datetime(df['time'])

# Vis de første radene
print(df.head())
print(f"\nAntall prognoser: {len(df)}")

---

### Oppgave 7b - Grunnleggende statistikk

Bruk pandas' innebygde metoder til å finne:
- Gjennomsnittstemperatur
- Gjennomsnittlig vindhastighet
- Minimum og maksimum temperatur

In [ ]:
# Skriv din kode her for å finne statistikk

# Tips: Bruk df.describe() eller df['kolonnenavn'].mean(), df['kolonnenavn'].min(), etc.


**Spørsmål:**
- Hva er gjennomsnittstemperaturen for de neste dagene?
- Hva er den høyeste og laveste temperaturen som er meldt?

_Skriv svaret ditt her_

***

## Gratulerer! 🎉

Du har nå:
- Lært hvordan man bruker et REST API
- Jobbet med JSON-data
- Laget en klasse for å strukturere data
- Brukt pandas til dataanalyse

### Bonusoppgaver (valgfritt)

Hvis du vil utfordre deg selv videre, prøv å:
1. Legge til flere metoder i Prognosis-klassen (f.eks. `get_humidity()`, `get_precipitation()`)
2. Lage et plott av temperatur over tid ved hjelp av matplotlib eller plotly
3. Sammenligne været for flere forskjellige lokasjoner
4. Hente data fra Yr sitt `complete` API i stedet for `compact` og se hva som er forskjellen

## Oppgave 6: Hent værprognose for flere timer

API-et gir oss prognoser for mange timer fremover. La oss lage en funksjon som henter prognoser for de neste timene.

In [ ]:
def get_hourly_forecast(lat, lon, hours=12):
    """
    Henter timesprognose for de neste X timene.
    
    Args:
        lat (float): Breddegrad
        lon (float): Lengdegrad
        hours (int): Antall timer å hente prognose for
    
    Returns:
        list: Liste med værprognoser
    """
    
    headers = {'User-Agent': 'VærApp-Student/1.0 (student@skole.no)'}
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"API-feil: Status {response.status_code}")
            return []
        
        data = response.json()
        timeseries = data['properties']['timeseries']
        
        forecast = []
        
        # Gå gjennom de første 'hours' prognosene
        for i, entry in enumerate(timeseries[:hours]):
            try:
                instant_data = entry['data']['instant']['details']
                
                forecast_entry = {
                    'tidspunkt': entry['time'],
                    'temperatur': instant_data['air_temperature'],
                    'luftfuktighet': instant_data['relative_humidity'],
                    'vindhastighet': instant_data['wind_speed']
                }
                
                forecast.append(forecast_entry)
                
            except KeyError:
                # Noen tidspunkter kan mangle data
                continue
        
        return forecast
        
    except Exception as e:
        print(f"Feil ved henting av prognose: {e}")
        return []

# Test funksjonen
oslo_forecast = get_hourly_forecast(59.9139, 10.7522, 8)

if oslo_forecast:
    print("📊 8-timers prognose for Oslo:")
    
    # Konverter til DataFrame for pen visning
    df = pd.DataFrame(oslo_forecast)
    
    # Forenkle tidspunkt-visning
    df['tid'] = pd.to_datetime(df['tidspunkt']).dt.strftime('%H:%M')
    
    # Vis de første prognosene
    print(df[['tid', 'temperatur', 'luftfuktighet', 'vindhastighet']].head(8).to_string(index=False))
else:
    print("❌ Kunne ikke hente værprognose")

**Oppgave 6a:** Finn temperatur-ekstremene i prognosen

In [ ]:
# Analyser temperaturen i prognosen
if oslo_forecast:
    temperatures = [entry['temperatur'] for entry in oslo_forecast]
    
    min_temp = min(temperatures)
    max_temp = max(temperatures)
    avg_temp = sum(temperatures) / len(temperatures)
    
    print(f"🌡️ Temperaturanalyse for de neste {len(oslo_forecast)} timene:")
    print(f"Laveste: {min_temp}°C")
    print(f"Høyeste: {max_temp}°C")
    print(f"Gjennomsnitt: {avg_temp:.1f}°C")
    print(f"Temperaturspenn: {max_temp - min_temp}°C")

## Oppgave 7: Sammenlign været i flere byer

La oss bruke funksjonene våre til å sammenligne været i flere norske byer.

In [ ]:
# Definer koordinater for norske byer
norske_byer = {
    'Oslo': (59.9139, 10.7522),
    'Bergen': (60.3913, 5.3221),
    'Trondheim': (63.4305, 10.3951),
    'Stavanger': (58.9700, 5.7331),
    'Tromsø': (69.6492, 18.9553)
}

# Hent værdata for alle byene
by_weather = []

print("🌍 Henter værdata for norske byer...")

for by_navn, (lat, lon) in norske_byer.items():
    print(f"  Henter data for {by_navn}...")
    
    weather = get_current_weather(lat, lon)
    
    if weather:
        weather['by'] = by_navn
        by_weather.append(weather)
    else:
        print(f"    ⚠️ Kunne ikke hente data for {by_navn}")

# Vis resultatene
if by_weather:
    print("\n🌤️ Værsammenligning:")
    
    # Konverter til DataFrame
    df = pd.DataFrame(by_weather)
    
    # Vis relevante kolonner
    weather_table = df[['by', 'temperatur_celsius', 'luftfuktighet_prosent', 'vindhastighet_ms']].round(1)
    weather_table.columns = ['By', 'Temperatur (°C)', 'Luftfuktighet (%)', 'Vind (m/s)']
    
    print(weather_table.to_string(index=False))
    
    # Finn ekstremene
    varmeste_idx = df['temperatur_celsius'].idxmax()
    kaldeste_idx = df['temperatur_celsius'].idxmin()
    
    print(f"\n🔥 Varmeste by: {df.loc[varmeste_idx, 'by']} ({df.loc[varmeste_idx, 'temperatur_celsius']}°C)")
    print(f"🧊 Kaldeste by: {df.loc[kaldeste_idx, 'by']} ({df.loc[kaldeste_idx, 'temperatur_celsius']}°C)")
else:
    print("❌ Kunne ikke hente værdata for noen byer")

**Oppgave 7a:** Finn byen med høyest vindhastighet

In [ ]:
# Din kode her:
# Finn byen med høyest vindhastighet
if by_weather:
    # vindigste_idx = df['vindhastighet_ms'].idxmax()
    # print(f"💨 Vindigste by: {df.loc[vindigste_idx, 'by']} ({df.loc[vindigste_idx, 'vindhastighet_ms']} m/s)")
    pass

## Oppgave 8: Avansert - Daglig værsammendrag

La oss lage en funksjon som gir oss et sammendrag av været for de neste dagene.

In [ ]:
def get_daily_summary(lat, lon, days=3):
    """
    Henter daglig værsammendrag for de neste dagene.
    
    Args:
        lat (float): Breddegrad
        lon (float): Lengdegrad
        days (int): Antall dager
    
    Returns:
        DataFrame: Daglig værsammendrag
    """
    
    # Hent prognose for mange timer (24 timer * antall dager)
    forecast = get_hourly_forecast(lat, lon, hours=days * 24)
    
    if not forecast:
        return None
    
    # Konverter til DataFrame
    df = pd.DataFrame(forecast)
    
    # Legg til dato-kolonne
    df['datetime'] = pd.to_datetime(df['tidspunkt'])
    df['dato'] = df['datetime'].dt.date
    
    # Gruppér etter dag og beregn statistikk
    daily_summary = df.groupby('dato').agg({
        'temperatur': ['min', 'max', 'mean'],
        'luftfuktighet': 'mean',
        'vindhastighet': ['mean', 'max']
    }).round(1)

    # Forenkle kolonnenavn
    daily_summary.columns = [
        'Min_temp', 'Max_temp', 'Snitt_temp',
        'Snitt_luftfuktighet', 'Snitt_vind', 'Max_vind'
    ]

    return daily_summary

# Test funksjonen for Oslo
print("📅 3-dagers værsammendrag for Oslo:")
oslo_summary = get_daily_summary(59.9139, 10.7522, 3)

if oslo_summary is not None:
    print(oslo_summary)

    print(f"📊 Analyse:")
    print(f"Varmeste dag: {oslo_summary['Max_temp'].max()}°C")
    print(f"Kaldeste dag: {oslo_summary['Min_temp'].min()}°C")
    print(f"Største temperaturspenn på en dag: {(oslo_summary['Max_temp'] - oslo_summary['Min_temp']).max()}°C")
else:
    print("❌ Kunne ikke hente daglig sammendrag")


## Bonusoppgaver

Hvis du har kommet så langt, kan du prøve disse utfordringsoppgavene!

### Bonusoppgave 1: Lag en værvarsel-klasse

Organiser funksjonaliteten i en Python-klasse for bedre struktur.

In [ ]:
class WeatherAPI:
    """Klasse for å hente værdata fra Yr.no API"""
    
    def __init__(self, user_agent="VærApp-Student/1.0 (student@skole.no)"):
        self.base_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
        self.headers = {'User-Agent': user_agent}
    
    def _make_request(self, lat, lon):
        """Hjelpemetode for å gjøre API-forespørsel"""
        url = f"{self.base_url}?lat={lat}&lon={lon}"
        
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()  # Kaster exception ved HTTP-feil
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API-feil: {e}")
            return None
    
    def get_current_weather(self, lat, lon):
        """Hent nåværende vær"""
        # Implementer denne metoden
        # Din kode her:
        pass
    
    def get_forecast(self, lat, lon, hours=12):
        """Hent timesprognose"""
        # Implementer denne metoden
        # Din kode her:
        pass

# Test klassen din
# weather_api = WeatherAPI()
# weather = weather_api.get_current_weather(59.9139, 10.7522)
# print(weather)

### Bonusoppgave 2: Robust feilhåndtering

Implementer en funksjon som prøver på nytt hvis API-et ikke svarer.

In [ ]:
import time

def robust_weather_request(lat, lon, max_retries=3, delay=1):
    """
    Robust API-forespørsel med retry-logikk.
    
    Args:
        lat, lon: Koordinater
        max_retries: Maksimalt antall forsøk
        delay: Sekunder å vente mellom forsøk
    """
    
    for attempt in range(max_retries):
        try:
            print(f"Forsøk {attempt + 1}/{max_retries}...")
            
            # Din kode her:
            # Implementer logikk som:
            # 1. Prøver å hente data
            # 2. Returnerer data hvis vellykket
            # 3. Venter 'delay' sekunder og prøver igjen ved feil
            # 4. Gir opp etter max_retries forsøk
            
            pass
            
        except Exception as e:
            print(f"Forsøk {attempt + 1} feilet: {e}")
            
            if attempt < max_retries - 1:
                print(f"Venter {delay} sekunder før nytt forsøk...")
                time.sleep(delay)
            else:
                print("Alle forsøk feilet!")
                return None

# Test funksjonen
# robust_data = robust_weather_request(59.9139, 10.7522)
# print(robust_data)

### Bonusoppgave 3: Værvarsel med nedbør

Yr.no API-et inneholder også informasjon om nedbør. Utforsk datastrukturen og hent ut nedbørsprognoser.

In [ ]:
# Utforsk 'next_1_hours' og 'next_6_hours' data
# Disse inneholder nedbørsprognoser

def get_precipitation_forecast(lat, lon):
    """Hent nedbørsprognose for de neste timene"""
    
    # Din kode her:
    # Tips: Se på 'next_1_hours' og 'next_6_hours' i datastrukturen
    # Disse inneholder 'precipitation_amount' og 'precipitation_amount_max'
    
    pass

# Test funksjonen
# precipitation = get_precipitation_forecast(59.9139, 10.7522)
# print(precipitation)

## Refleksjonsspørsmål

Ta deg tid til å tenke over det du har lært:

1. **API-forståelse**: Hva er forskjellen mellom et API og en vanlig nettside?

2. **Feilhåndtering**: Hvorfor er det viktig å håndtere feil når man arbeider med API-er?

3. **Datastruktur**: Yr.no returnerer mye data. Hvordan bestemte du hvilke verdier som var viktigst å hente ut?

4. **Etikk**: Hvilke etiske hensyn bør man ta når man bruker andres API-er?

5. **Utvidelser**: Hvilke andre funksjoner kunne du tenke deg å legge til i en værapp?

Skriv ned tankene dine i cellen under:

**Mine refleksjoner:**

<!-- Skriv dine tanker her -->



## Oppsummering

🎉 **Gratulerer! Du har nå lært:**

✅ **REST API-konsepter**
- Hva et API er og hvordan det fungerer
- HTTP-metoder (GET, POST, etc.)
- Status koder (200, 404, etc.)

✅ **requests-biblioteket**
- Gjøre HTTP-forespørsler
- Håndtere JSON-data
- Sette headers og parametere

✅ **Yr.no API**
- Hente værdata for spesifikke koordinater
- Forstå komplekse datastrukturer
- Hente både nåværende vær og prognoser

✅ **Databehandling**
- Pakke ut relevant informasjon fra API-svar
- Sammenligne data fra flere kilder
- Lage sammendrag og statistikk

✅ **Feilhåndtering**
- Try/except blokker
- Håndtere nettverksfeil
- Validere data

### Neste steg i læringen:

🔗 **Utforsk andre API-er:**
- [JSONPlaceholder](https://jsonplaceholder.typicode.com/) - Test-API
- [OpenWeatherMap](https://openweathermap.org/api) - Alternativt vær-API
- [REST Countries](https://restcountries.com/) - Landeinformasjon

📚 **Lær mer om:**
- API-autentisering (API-nøkler, OAuth)
- GraphQL (alternativ til REST)
- Rate limiting og caching
- Asynkron programmering med `asyncio`

🛠️ **Prosjektideer:**
- Bygg en værapp med grafisk brukergrensesnitt
- Lag en bot som sender daglige værmeldinger
- Kombiner flere API-er (vær + kart + nyheter)
- Lag en værlogger som lagrer data i en database

### Viktige ressurser:

- 📖 [Requests dokumentasjon](https://docs.python-requests.org/)
- 🌤️ [Yr.no API dokumentasjon](https://api.met.no/weatherapi/)
- 🔧 [HTTP statuskoder](https://httpstatuses.com/)
- 📊 [Pandas dokumentasjon](https://pandas.pydata.org/docs/)

**Lykke til med videre utforskning av API-verdenen! 🚀**